In [ ]:
import matplotlib.pyplot as plt
import polars as pl

from src.Parametric_UMAP import PUMAP, load_pumap
import torch.nn.functional as F
import torch

In [ ]:
PATH_SAVE_LD = 'src/Results_Uniform/Low Dimension'
PATH_SAVE_D = 'src/Results_Uniform/Density'
PATH_RESULTS = 'src/Results_Uniform'
PATH_DATA = '/home/nicolasdc/Documents/Datasets/Photon-Number-Classification/NIST CSV'

data_train = pl.read_csv(f'{PATH_DATA}/uniform_train.csv', has_header=False, separator=",").to_numpy()
data_test = pl.read_csv(f'{PATH_DATA}/uniform_test.csv', has_header=False, separator=",").to_numpy()
expected_prob = pl.read_csv(f'{PATH_DATA}/uniform_expected_prob.csv', has_header=False, separator=",").to_numpy()
db_train = pl.read_csv(f'{PATH_DATA}/db_train.csv', has_header=False, separator=",").to_numpy()
db_test = pl.read_csv(f'{PATH_DATA}/db_test.csv', has_header=False, separator=",").to_numpy()

In [ ]:
pumap = PUMAP(
        encoder=None,         
        decoder=None,          
        n_neighbors=30, #30
        min_dist=0.15, #0.15
        metric="euclidean",
        n_components=1,
        beta=0, # How much to weigh reconstruction loss for decoder
        reconstruction_loss=F.binary_cross_entropy_with_logits, # pass in custom reconstruction loss functions
        random_state=None,
        lr=1e-4,
        epochs = 50,
        batch_size=5000, 
        num_workers=31,
        num_gpus=1,
        match_nonparametric_umap=False # Train network to match embeddings from non parametric umap
)

pumap.fit(torch.from_numpy(data_train).view(-1,350).to(dtype=torch.float))

In [ ]:
model = load_pumap(f'lightning_logs/version_2/checkpoints/epoch=29-step=210.ckpt').to('cpu')


embedding = model.encoder(torch.from_numpy(data_test).view(-1,350).to(dtype=torch.float)) # (50000, 2)

plt.hist(embedding.detach().numpy().flatten(), bins=1000)
plt.yscale('log')
plt.show()